## Iteration 5

Starting from iteration 1 and then converting the AVProduct States Identifier into its bitwise representation.

In [259]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier


In [260]:
file="50ksample.csv"

In [261]:
data_df=pd.read_csv(file)
data_df.head()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,1514734,2b7b688e4a4315bc4f21f6849d03ab66,win8defender,1.1.15200.1,4.11.15063.0,1.275.828.0,0,7.0,0,NaN,...,33968.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
1,2926031,53f3fd511ba64acaec4e0c8559143301,win8defender,1.1.15200.1,4.18.1807.18075,1.275.727.0,0,7.0,0,NaN,...,19951.0,0,0.0,0.0,0,0,0.0,1.0,5.0,0
2,7189617,ce48198690d1e4b437ffb60bfcb9e8df,win8defender,1.1.15100.1,4.18.1807.18075,1.273.516.0,0,7.0,0,NaN,...,33066.0,0,NaN,0.0,0,0,0.0,1.0,13.0,1
3,6264912,b3cfee470ee30c729d3a33f0017a8b51,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,...,12303.0,1,NaN,0.0,0,0,0.0,0.0,3.0,0
4,3321923,5f4f1fa2419439ff865d42a2a06ac8b5,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1486.0,0,7.0,0,NaN,...,4357.0,0,NaN,0.0,0,1,0.0,0.0,3.0,1


In [262]:
data_df=data_df[["EngineVersion","AVProductStatesIdentifier","Processor","SmartScreen","Census_MDC2FormFactor","Platform","OsBuild", "Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer","HasDetections"]]

data_df.head()


,EngineVersion,AVProductStatesIdentifier,Processor,SmartScreen,Census_MDC2FormFactor,Platform,OsBuild,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,HasDetections
0,1.1.15200.1,53447.0,x64,NaN,Desktop,windows10,15063,0.0,0.0,0
1,1.1.15200.1,53447.0,x64,RequireAdmin,Desktop,windows10,16299,0.0,1.0,0
2,1.1.15100.1,53447.0,x64,NaN,Notebook,windows10,15063,0.0,1.0,1
3,1.1.14600.4,47238.0,x64,NaN,Notebook,windows10,17134,0.0,0.0,0
4,1.1.15200.1,53447.0,x64,NaN,Desktop,windows10,17134,0.0,0.0,1


Drop that have nulls in AVProductStatesIdentifier

AVProductsEnabled fill nulls with 0
Browser Identtifier needs to be filled wiht 0
SmartScreen: clean up off and Off, on and On, “&#x02” and “&#x01" will have own columns., null should go to off bucket
Census_IsAlwaysConnectedCapable: replace with 0
Wdft_IsGamer: replace with 0



In [263]:
data_df[["Census_IsAlwaysOnAlwaysConnectedCapable","Wdft_IsGamer"]].fillna(value=0, inplace=True)
data_df["SmartScreen"].fillna(value="off", inplace=True)
data_df["SmartScreen"].replace("Off", "off", inplace=True)
data_df["SmartScreen"].replace("On", "on", inplace=True)
data_df["SmartScreen"].fillna(value="off", inplace=True)
data_df["AVProductStatesIdentifier"].fillna(value="0", inplace=True)
## This drops any remaining nulls 
data_df.dropna(how='any',inplace=True)


/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [264]:
# Convert the float in the AVProductStatesIdentifier feature to a binary
data_df['AVProductStatesIdentifier'] = data_df.AVProductStatesIdentifier.apply(lambda x: format(int(x), '04b'))

In [265]:
# Convert the Binary to a string
data_df['AVProductStatesIdentifier']=data_df['AVProductStatesIdentifier'].astype(str)

In [266]:
# Add Leading 0's to the sting
data_df["AVProductStatesIdentifier"]=data_df["AVProductStatesIdentifier"].str.zfill(17)

In [267]:
data_df.head()

,EngineVersion,AVProductStatesIdentifier,Processor,SmartScreen,Census_MDC2FormFactor,Platform,OsBuild,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,HasDetections
0,1.1.15200.1,01101000011000111,x64,off,Desktop,windows10,15063,0.0,0.0,0
1,1.1.15200.1,01101000011000111,x64,RequireAdmin,Desktop,windows10,16299,0.0,1.0,0
2,1.1.15100.1,01101000011000111,x64,off,Notebook,windows10,15063,0.0,1.0,1
3,1.1.14600.4,01011100010000110,x64,off,Notebook,windows10,17134,0.0,0.0,0
4,1.1.15200.1,01101000011000111,x64,off,Desktop,windows10,17134,0.0,0.0,1


In [268]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47960 entries, 0 to 49999
Data columns (total 10 columns):
EngineVersion                              47960 non-null object
AVProductStatesIdentifier                  47960 non-null object
Processor                                  47960 non-null object
SmartScreen                                47960 non-null object
Census_MDC2FormFactor                      47960 non-null object
Platform                                   47960 non-null object
OsBuild                                    47960 non-null int64
Census_IsAlwaysOnAlwaysConnectedCapable    47960 non-null float64
Wdft_IsGamer                               47960 non-null float64
HasDetections                              47960 non-null int64
dtypes: float64(2), int64(2), object(6)
memory usage: 4.0+ MB


In [269]:
# Make a new dataframe that's the bitwise representation of the string
AVProduct_df=data_df['AVProductStatesIdentifier'].str.split(pat="",n=-1,expand=True)
AVProduct_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,
1,,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,
2,,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,
3,,0,1,0,1,1,1,0,0,0,1,0,0,0,0,1,1,0,
4,,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,


In [270]:
for n in range(1 , 18):
    data_df[f"AV_states_{n}"]=AVProduct_df[n].astype(int)

In [271]:
data_df=data_df.drop(["AVProductStatesIdentifier"],axis=1)

In [272]:
y=data_df["HasDetections"]
len(y)

47960

In [273]:
X=pd.get_dummies(data_df)
X=X.drop(["HasDetections"],axis=1)
dummy_columns=X.columns.get_values()
dummy_columns=dummy_columns.tolist()

X.head()

,OsBuild,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,AV_states_1,AV_states_2,AV_states_3,AV_states_4,AV_states_5,AV_states_6,AV_states_7,...,Census_MDC2FormFactor_MediumServer,Census_MDC2FormFactor_Notebook,Census_MDC2FormFactor_PCOther,Census_MDC2FormFactor_ServerOther,Census_MDC2FormFactor_SmallServer,Census_MDC2FormFactor_SmallTablet,Platform_windows10,Platform_windows2016,Platform_windows7,Platform_windows8
0,15063,0.0,0.0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
1,16299,0.0,1.0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,15063,0.0,1.0,0,1,1,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0
3,17134,0.0,0.0,0,1,0,1,1,1,0,...,0,1,0,0,0,0,1,0,0,0
4,17134,0.0,0.0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [274]:
dummy_columns

['OsBuild',
 'Census_IsAlwaysOnAlwaysConnectedCapable',
 'Wdft_IsGamer',
 'AV_states_1',
 'AV_states_2',
 'AV_states_3',
 'AV_states_4',
 'AV_states_5',
 'AV_states_6',
 'AV_states_7',
 'AV_states_8',
 'AV_states_9',
 'AV_states_10',
 'AV_states_11',
 'AV_states_12',
 'AV_states_13',
 'AV_states_14',
 'AV_states_15',
 'AV_states_16',
 'AV_states_17',
 'EngineVersion_1.1.10302.0',
 'EngineVersion_1.1.11701.0',
 'EngineVersion_1.1.12101.0',
 'EngineVersion_1.1.12603.0',
 'EngineVersion_1.1.12805.0',
 'EngineVersion_1.1.12902.0',
 'EngineVersion_1.1.13000.0',
 'EngineVersion_1.1.13103.0',
 'EngineVersion_1.1.13202.0',
 'EngineVersion_1.1.13303.0',
 'EngineVersion_1.1.13407.0',
 'EngineVersion_1.1.13504.0',
 'EngineVersion_1.1.13601.0',
 'EngineVersion_1.1.13701.0',
 'EngineVersion_1.1.13704.0',
 'EngineVersion_1.1.13802.0',
 'EngineVersion_1.1.13803.0',
 'EngineVersion_1.1.13804.0',
 'EngineVersion_1.1.13903.0',
 'EngineVersion_1.1.14003.0',
 'EngineVersion_1.1.14104.0',
 'EngineVersion_1

In [275]:

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [276]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [277]:

print(f"Training: {rf.score(X_train, y_train)}")
print(f"Testing: {rf.score(X_test, y_test)}" )



Training: 0.7119266055045872
Testing: 0.6014178482068391


In [278]:
importances = rf.feature_importances_
sorted(zip(rf.feature_importances_, dummy_columns), reverse=True)

[(0.18709668737159305, 'OsBuild'),
 (0.10620617035751705, 'SmartScreen_ExistsNotSet'),
 (0.05276705310309367, 'Wdft_IsGamer'),
 (0.0382866245968496, 'SmartScreen_off'),
 (0.03231612098360244, 'SmartScreen_RequireAdmin'),
 (0.03140237849626646, 'EngineVersion_1.1.15100.1'),
 (0.028084655715641213, 'EngineVersion_1.1.15200.1'),
 (0.026725482561495265, 'Census_MDC2FormFactor_Notebook'),
 (0.02158841342408118, 'Census_MDC2FormFactor_Desktop'),
 (0.018987826721724742, 'AV_states_3'),
 (0.018270719422572218, 'AV_states_10'),
 (0.01752245715019306, 'AV_states_9'),
 (0.017378878371421033, 'AV_states_16'),
 (0.016787307907097204, 'AV_states_15'),
 (0.01659399933193075, 'AV_states_12'),
 (0.01654160725713749, 'AV_states_7'),
 (0.015550242771109071, 'Census_IsAlwaysOnAlwaysConnectedCapable'),
 (0.015243380668398641, 'AV_states_2'),
 (0.015202679686083666, 'AV_states_6'),
 (0.015177652116534395, 'EngineVersion_1.1.15000.2'),
 (0.015125957586752524, 'AV_states_14'),
 (0.014751641094232609, 'Census_

In [233]:
from sklearn.metrics import classification_report
print(classification_report(y_test, rf.predict(X_test),
                            ))

             precision    recall  f1-score   support

          0       0.56      0.64      0.60      5947
          1       0.59      0.51      0.55      6043

avg / total       0.58      0.57      0.57     11990



## Iteration 6

Iterations 6 and 7 are comparing the performance of bitwise expansion of AVProductStatesIdentifier and AVProductStatesIdentifier as a pain integer.

I mislabeled the variable and it is iter5.

In [312]:
data_df=pd.read_csv(file)

iter5=data_df.copy()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [313]:
iter5.head()

,Unnamed: 0,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,1514734,2b7b688e4a4315bc4f21f6849d03ab66,win8defender,1.1.15200.1,4.11.15063.0,1.275.828.0,0,7.0,0,NaN,...,33968.0,0,NaN,0.0,0,0,0.0,0.0,3.0,0
1,2926031,53f3fd511ba64acaec4e0c8559143301,win8defender,1.1.15200.1,4.18.1807.18075,1.275.727.0,0,7.0,0,NaN,...,19951.0,0,0.0,0.0,0,0,0.0,1.0,5.0,0
2,7189617,ce48198690d1e4b437ffb60bfcb9e8df,win8defender,1.1.15100.1,4.18.1807.18075,1.273.516.0,0,7.0,0,NaN,...,33066.0,0,NaN,0.0,0,0,0.0,1.0,13.0,1
3,6264912,b3cfee470ee30c729d3a33f0017a8b51,win8defender,1.1.14600.4,4.13.17134.1,1.263.48.0,0,7.0,0,NaN,...,12303.0,1,NaN,0.0,0,0,0.0,0.0,3.0,0
4,3321923,5f4f1fa2419439ff865d42a2a06ac8b5,win8defender,1.1.15200.1,4.18.1807.18075,1.275.1486.0,0,7.0,0,NaN,...,4357.0,0,NaN,0.0,0,1,0.0,0.0,3.0,1


In [314]:
iter5= iter5[['AVProductStatesIdentifier',"HasDetections"]]
iter5.head()

,AVProductStatesIdentifier,HasDetections
0,53447.0,0
1,53447.0,0
2,53447.0,1
3,47238.0,0
4,53447.0,1


In [315]:
iter5["AVProductStatesIdentifier"]=iter5["AVProductStatesIdentifier"].fillna(value=0)
X=pd.get_dummies(iter5)
X=X.drop(["HasDetections"],axis=1)
dummy_columns=X.columns.get_values()
dummy_columns=dummy_columns.tolist()

X.head()

,AVProductStatesIdentifier
0,53447.0
1,53447.0
2,53447.0
3,47238.0
4,53447.0


In [316]:
y=data_df["HasDetections"]
len(y)

50000

In [317]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [318]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [319]:

print(f"Training: {rf.score(X_train, y_train)}")
print(f"Testing: {rf.score(X_test, y_test)}" )

Training: 0.59272
Testing: 0.57712


In [320]:
for i  in range(len(dummy_columns)):
    dummy_columns[i]=str(dummy_columns[i])

In [321]:
importances = rf.feature_importances_
sorted(zip(rf.feature_importances_, dummy_columns), reverse=True)

[(1.0, 'AVProductStatesIdentifier')]

## Iteration 7

Checking AVProductStatesIdentifier as its bitwise reprensentation:



In [333]:
iter6=pd.read_csv(file)


/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [334]:
iter6= iter6[['AVProductStatesIdentifier',"HasDetections"]]
iter6.head()

,AVProductStatesIdentifier,HasDetections
0,53447.0,0
1,53447.0,0
2,53447.0,1
3,47238.0,0
4,53447.0,1


In [335]:
iter6["AVProductStatesIdentifier"].fillna(value="0", inplace=True)
## This drops any remaining nulls 
iter6.dropna(how='any',inplace=True)

In [336]:
y=iter6["HasDetections"]
len(y)

50000

In [337]:
# Convert the float in the AVProductStatesIdentifier feature to a binary
iter6['AVProductStatesIdentifier'] = iter6.AVProductStatesIdentifier.apply(lambda x: format(int(x), '04b'))
# Convert the Binary to a string
iter6['AVProductStatesIdentifier']=iter6['AVProductStatesIdentifier'].astype(str)
# Add Leading 0's to the sting
iter6["AVProductStatesIdentifier"]=iter6["AVProductStatesIdentifier"].str.zfill(17)
# Make a new dataframe that's the bitwise representation of the string
AVProduct_df=iter6['AVProductStatesIdentifier'].str.split(pat="",n=-1,expand=True)
AVProduct_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,
1,,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,
2,,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,
3,,0,1,0,1,1,1,0,0,0,1,0,0,0,0,1,1,0,
4,,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1,


In [338]:
# Add the individual columns to the dataframe
for n in range(1 , 18):
    iter6[f"AV_states_{n}"]=AVProduct_df[n].astype(str)
    
# Drop the original column
iter6=iter6.drop(["AVProductStatesIdentifier"],axis=1)
iter6.head()

,HasDetections,AV_states_1,AV_states_2,AV_states_3,AV_states_4,AV_states_5,AV_states_6,AV_states_7,AV_states_8,AV_states_9,AV_states_10,AV_states_11,AV_states_12,AV_states_13,AV_states_14,AV_states_15,AV_states_16,AV_states_17
0,0,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1
1,0,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1
2,1,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1
3,0,0,1,0,1,1,1,0,0,0,1,0,0,0,0,1,1,0
4,1,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1


In [339]:
y=iter6["HasDetections"]
len(y)

50000

In [340]:
X=pd.get_dummies(iter6, drop_first=True)
X=X.drop(["HasDetections"],axis=1)
dummy_columns=X.columns.get_values()
dummy_columns=dummy_columns.tolist()

X.head()

,AV_states_1_1,AV_states_2_1,AV_states_3_1,AV_states_4_1,AV_states_5_1,AV_states_6_1,AV_states_7_1,AV_states_8_1,AV_states_9_1,AV_states_10_1,AV_states_11_1,AV_states_12_1,AV_states_13_1,AV_states_14_1,AV_states_15_1,AV_states_16_1,AV_states_17_1
0,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1
1,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1
2,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1
3,0,1,0,1,1,1,0,0,0,1,0,0,0,0,1,1,0
4,0,1,1,0,1,0,0,0,0,1,1,0,0,0,1,1,1


In [341]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)

In [342]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)

In [343]:

print(f"Training: {rf.score(X_train, y_train)}")
print(f"Testing: {rf.score(X_test, y_test)}" )

Training: 0.59272
Testing: 0.57944
